In [ ]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import tensorflow as tf
import numpy as np
import tflearn
import random
import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
from google.colab import files
files.upload()

In [ ]:
!pip install DNN

In [ ]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import tensorflow as tf
import numpy as np
import tflearn
import random
import json
import pickle
import DNN

import requests
response = requests.get('https://example.com/intents.json')
intents = json.loads(response.content)

words = []
classes = []
documents = []
ignore = ['?']
data_file = open('intents.json', encoding="utf8").read()
intents = json.loads(data_file)

for intent in intents['intents']:
  for pattern in intent['patterns']:
    w = nltk.word_tokenize(pattern)
    words.extend(w)
    documents.append((w, intent['tag']))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

pickle.dump(words,open('texts1.pkl','wb'))
pickle.dump(classes,open('labels1.pkl','wb'))

training = []
output = []

output_empty = [0]*len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data Created")

tf.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
model.fit(train_x, train_y, n_epoch=100, batch_size=8, show_metric=True)
model.save('model.tflearn')
print("model created")


nltk.download('popular')

import pickle
import numpy as np

model.load('./model.tflearn')
import json
import random
intents = json.loads(open('intents.json', encoding="utf8").read())
words = pickle.load(open('texts1.pkl','rb'))
classes = pickle.load(open('labels1.pkl','rb'))

def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
  return sentence_words

def bow(sentence, words, show_details=False):
  sentence_words = clean_up_sentence(sentence)
  bag = [0]*len(words)
  for s in sentence_words:
    for i,w in enumerate(words):
      if w == s:
        bag[i] = 1
        if show_details:
          print("found in bag: %s" % w)

  return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res


from flask import Flask, render_template, request

app = Flask(__name__)
app.static_folder = 'static'

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/get")
def get_bot_response():
    userText = request.args.get('msg')
    return chatbot_response(userText)


if __name__ == "__main__":
    app.run(debug = False, host = '0.0.0.0', port = '5000')


In [ ]:
with open('intents.json') as json_data:
  intents = json.load(json_data)

In [ ]:
words = []
classes = []
documents = []
ignore = ['?']
for intent in intents['intents']:
  for pattern in intent['patterns']:
    w = nltk.word_tokenize(pattern)
    words.extend(w)
    documents.append((w, intent['tag']))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

In [ ]:
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

963 documents
210 classes [' Global_Industry_Classification_Standard', ' financial statements', 'Common Stock ', 'Corporation', 'Financial_Industry_Regulatory_Authority', 'Government_stability', 'Index', 'Over_the_Counter', 'Risk_lover ', 'Sectors_are_there_to_invest_in_Stock_Market ', 'Self_Regulatory_Organization', '_DVR_shares_', '_FINRA_financial_SRO_', '_HDFC-stocks-failing_', '_IPO_', '_IPO_investment_right_for_an_investor_', '_IPO_works_in_companies', '_Invest_ment', '_Investment_analyst_', '_OTC_stocks_publicly_traded_', '_advanatages_of_derivatives_', '_advantages_of_delisting_', '_advantages_of_nsdl_', '_bank_nifty_', '_beginners-buy-stocks_', '_benefits_of_nse_', '_bid_in_ipo_', '_bigger_NSE_or_BSE_', '_biggest_ipo_in_india_', '_bombay_stock_exchange_', '_broker_dealer_', '_buy-hdfc-stock_', '_buy-share-any-time_', '_cancel-multiple-orders_', '_change_in_budget_', '_charges_and_taxes_on_derivative_contracts_', '_check-my-margin-details_', '_common_stocks_referred_by_companie

In [ ]:
import pickle
pickle.dump(words,open('texts1.pkl','wb'))
pickle.dump(classes,open('labels1.pkl','wb'))

In [ ]:
training = []
output = []

output_empty = [0]*len(classes)

for doc in documents:
  bag = []
  pattern_words = doc[0]
  pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)

  output_row = list(output_empty)
  output_row[classes.index(doc[1])] = 1

  training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])

<ipython-input-9-5c442f400a69>:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [ ]:
tf.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.fit(train_x, train_y, n_epoch=200, batch_size=8, show_metric=True)
model.save('model.tflearn')


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Streaming output truncated to the last 5000 lines.
Training Step: 21712  | total loss: 0.01352 | time: 0.150s
| Adam | epoch: 180 | loss: 0.01352 - acc: 0.9923 -- iter: 424/963
Training Step: 21713  | total loss: 0.01222 | time: 0.154s
| Adam | epoch: 180 | loss: 0.01222 - acc: 0.9930 -- iter: 432/963
Training Step: 21714  | total loss: 0.01101 | time: 0.160s
| Adam | epoch: 180 | loss: 0.01101 - acc: 0.9937 -- iter: 440/963
Training Step: 21715  | total loss: 0.00993 | time: 0.165s
| Adam | epoch: 180 | loss: 0.00993 - acc: 0.9944 -- iter: 448/963
Training Step: 21716  | total loss: 0.00895 | time: 0.169s
| Adam | epoch: 180 | loss: 0.00895 - acc: 0.9949 -- iter: 456/963
Training Step: 21717  | total loss: 0.00807 | time: 0.172s
| Adam | epoch: 180 | loss: 0.00807 - acc: 0.9954 -- iter: 464/963
Training Step: 21718  | total loss: 0.00731 | time: 0.175s
| Adam | epoch: 180 | loss: 0.00731 - acc: 0.9959 -- iter: 472/963
Training Step: 21719  | total loss: 0.00707 | time: 0.178s
| Adam |

In [ ]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open("training_data", "wb"))

In [ ]:
data = pickle.load( open( "training_data", "rb") )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [ ]:
with open('intents.json') as json_data:
  intents = json.load(json_data)

In [ ]:
model.load('./model.tflearn')

In [ ]:
def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
  return sentence_words

def bow(sentence, words, show_details=False):
  sentence_words = clean_up_sentence(sentence)
  bag = [0]*len(words)
  for s in sentence_words:
    for i,w in enumerate(words):
      if w == s:
        bag[i] = 1
        if show_details:
          print("found in bag: %s" % w)

  return(np.array(bag))

In [ ]:
context ={}

ERROR_THRESHOLD = 0.25
def classify(sentence):
  results = model.predict([bow(sentence, words)])[0]
  results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
  results.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in results:
    return_list.append((classes[r[0]], r[1]))
  return return_list

def response(sentence, userID='123', show_details=False):
  results = classify(sentence)
  if results:
    while results:
      for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return print(random.choice(i['responses']))

      results.pop(0)

In [ ]:
classify('What is Stock?')

[('stock', 0.9956429)]

In [ ]:
response('stock')

A stock is a type of investment in a company. It is also known as equity, is a security that represents the ownership of a fraction of the issuing corporation.
 Also we can say, A stock is a general term used to describe the ownership certificates of any company.
 you can find more info on this link: https://economictimes.indiatimes.com/definition/stocks


In [ ]:
response('shares')

A share, on the other hand, refers to the stock certificate of a particular company. Holding a particular company's share makes you a shareholder.
 for more info: https://www.investopedia.com/terms/s/shares.asp


In [ ]:
response('hello', show_details=True)

tag: greetings
What Can I Do For You?
